# The Adapter pattern

The flyweight pattern is designed for conserving memory

In [1]:
class AgeCalculator:
    def __init__(self, birthday):
        self.year, self.month, self.day = (int(x) for x in birthday.split('-'))
    
    def calculate_age(self, date):
        year, month, day = (int(x) for x in date.split('-'))
        age = year - self.year
        if (month,day) < (self.month, self.day):
            age -= 1
        return age

In [2]:
#Instead, we can write an adapter that allows a normal date to be plugged into a
# normal AgeCalculator class:
import datetime
class DateAgeAdapter:
    def _str_date(self, date):
        return date.strftime("%Y-%m-%d")
    
    def __init__(self, birthday):
        birthday = self._str_date(birthday)
        self.calculator = AgeCalculator(birthday)
        
    def get_age(self, date):
        date = self._str_date(date)
        return self.calculator.calculate_age(date)


In [3]:
import datetime
class AgeableDate(datetime.date):
    def split(self, char):
        return self.year, self.month, self.day


In [4]:
bd = AgeableDate(1974, 6, 13)
today = AgeableDate.today()

In [5]:
today

AgeableDate(2017, 12, 21)

In [6]:
a = AgeCalculator(bd)
a.calculate_age(today)

43

# The facade pattern

In [8]:
# lets built a application to send and get emails using the facade pattern to abstract a complex taks intoa  simplier interface
import smtplib
import imaplib

class EmailFacade:
    def __init__(self, host, username, password):
        self.host = host
        self.username = username
        self.password = password
    
    def send_email(self, to_email, subject, message):
        if not "@" in self.username:
            from_email = "{0}@{1}".format(self.username, self.host)
        else:
            from_email = self.username
        message = ("From: {0}\r\n"
                  "To: {1}\r\n"
                  "Subject: {2}\r\n\r\n{3}").format(from_email, to_email, subject, message)
        smtp = smtplib.SMTP(self.host)
        smtp.login(self.username, self.password)
        smtp.sendmail(from_email, [to_email,message])
    
    def get_inbox(self):
        mailbox = imaplib.IMAP4(self.host)
        mailbox.login(bytes(self.username, 'utf8'), bytes(self.password, 'utf8'))
        mailbox.select()
        x, data = mailbox.search(None, 'ALL')
        messages = []
        for num in data[0].split():
            x, message = mailbox.fetch(num, '(RFC822)')
            messages.append(message[0][1])
        return messages
    
        

# The flyweight pattern

In [9]:
# for this pattern we will use a dictionary type from a module that will delete the keys that doesnt have a value
import weakref

class CarModel:
    _models = weakref.WeakValueDictionary()
    
    def __new__(cls, model_name, *args, **kwargs):
        model = cls._models.get(model_name)
        if not model:
            model = super().__new__(cls)
            cls._models[model_name] = model
        return model
    
    def __init__(self, model_name, air=False, tilt=False, cruise_control=False,
                 power_locks=False, allow_wheels=False, usb_charger=False):
        if not hasattr(self, "initted"):
            self.initted = True
            self.model_name = model_name
            self.air = air
            self.tilt = tilt
            self.cruise_control = cruise_control
            self.power_locks = power_locks
            self.allow_wheels = allow_wheels
            self.usb_charger = usb_charger
            
    def check_serial(self, serial_number):
        print("Sorry, we are unable to check "
                 "the serial number {0} on the {1} "
                 "at this time".format(serial_number, self.model_name))
        

In [10]:
class Car:
    def __init__(self, model, color, serial):
        self.model = model
        self.color = color
        self.serial = serial
        
    def check_serial(self):
        return self.mode.check_serial(self.serial)
    

In [11]:
dx = CarModel("FIT DX")
lx = CarModel("FIT LX", air=True, cruise_control=True, power_locks=True, tilt=True)
car1 = Car(dx, "blue", "1235")
car2 = Car(dx, "black", "1236")
car3 = Car(lx, "red", "1237")


In [12]:
id(lx)

1884780667232

In [13]:
del lx

In [14]:
del car3

In [15]:
import gc
gc.collect()

576

In [16]:
lx = CarModel("FIT LX", air=True, cruise_control=True, power_locks=True, tilt=True)

In [17]:
id(lx)

1884777505792

In [18]:
lx  = CarModel("FIT LX")

In [19]:
id(lx)
# should get the same id as just above

1884777505792

In [20]:
lx.air

True

# The command pattern

#an example of this pattern is used in the GUI actions frameworks. how to deal with actions and invokers and receives.
Let's implement a simple command pattern that provides commands for Save and
Exit actions. We'll start with some modest receiver classes:

In [1]:
import sys

class Window:
    def exit(self):
        sys.exit(0)

class Document:
    def __init__(self, filename):
        self.filename = filename
        self.contents = "This file cannot be modified"
    def save(self):
        with open(self.filename, 'w') as file:
            file.write(self.contents)
            

In [2]:
# the invokers interfaces
class ToolbarButton:
    def __init__(self, name, iconname):
        self.name = name
        self.iconname = iconname
    
    def click(self):
        self.command.execute()

class MenuItem:
    def __init__(self, menu_name, menuitem_name):
        self.menu = menu_name
        self.item = menuitem_name
    
    def click(self):
        self.command.execute()

class KeyboardShortcut:
    def __init__(self, key, modifier):
        self.key = key
        self.modifier = modifier
    
    def keypress(self):
        self.command.execute()
    

In [8]:
# LETS SETS THE COMMANDS
class SaveCommand:
    def __init__(self, document):
        self.document = document
    
    def execute(self):
        self.document.save()

class ExitCommand:
    def __init__(self, window):
        self.window = window
    
    def execute(self):
        self.window.exit()
        

In [13]:
#testing
window = Window()
document = Document("chapter11\\a_document.txt")
save = SaveCommand(document)
exit = ExitCommand(window)

save_button = ToolbarButton('save','chapter11\\save.png')
save_button.command = save
save_keystroke = KeyboardShortcut("s", "ctrl")
save_keystroke.command = save
exit_menu = MenuItem("File","Exit")
exit_menu.command = exit


In [14]:
exit_menu.click()

SystemExit: 0

C:\Users\XI316270\Applications\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [15]:
save_keystroke.keypress()

#Instead of an object with an execute() method, we can write a function
and use that as the command directly. This is a common paradigm for the command
pattern in Python:

In [16]:
import sys

class Window:
    def exit(self):
        sys.exit(0)

class MenuItem:
    def click(self):
        self.command()

window = Window()
menu_item = MenuItem()
menu_item.command = window.exit


In [ ]:
#above example with a function is the simplier way
# Inc ase we want to maintain a state for instance, we then need classes
# but we can add a __call__ method to the class, so it acts like a funciton
# A more advanced usage we can use this:
class Document:
    def __init(self, filename):
        self.filename = filename
        self.contents = "This file cannot be modified"
        
    def save(self):
        with open(self.filename, 'w') as file:
            file.write(self.contents)
            
class KeyboardShortcut:
    def keypress(self):
        self.command()

class SaveCommand:
    def __init__(self, document):
        self.document = document
    
    def __call__(self):
        self.document.save()

document = Document("chapter11\\a_file.txt")
shortcut = KeyboardShortcut()
save_command = SaveCommand(document)
shortcut.command = save_command

            

# The abstract factory pattern

A pattern to design classes for formating dates and currency according to locate countries, like Usa and France

In [5]:
class FranceDateFormatter:
    def format_date(self, y, m, d):
        y, m, d = (str(x) for x in (y,m,d))
        y = '20' + y if len(y) == 2 else y
        m = '0' + m if len(m) == 1 else m
        d = '0' + d if len(d) == 1 else d
        return("{0}/{1}/{2}".format(d,m,y))

class USADateFormatter:
    def format_date(self, y, m, d):
        y, m, d = (str(x) for x in (y,m,d))
        y = '20' + y if len(y) == 2 else y
        m = '0' + m if len(m) == 1 else m
        d = '0' + d if len(d) == 1 else d
        return "{0}-{1}-{2}".format(m,d,y)

class FranceCurrencyFormatter:
    def format_currency(self, base, cents):
        base, cents = (str(x) for x in (base, cents))
        if len(cents) == 0:
            cents = '00'
        elif len(cents) == 1:
            cents = '0' + cents
        digits = []
        for i,c in enumerate(reversed(base)):
            if i and not i % 3:
                digits.append(' ')
            digits.append(c)
        base = ''.join(reversed(digits))
        return "{0}E{1}".format(base, cents)

class USACurrencyFormatter:
    def format_currency(self, base, cents):
        base, cents = (str(x) for x in (base, cents))
        if len(cents) == 0:
            cents = '00'
        elif len(cents) == 1:
            cents = '0' + cents
        digits = []
        for i,c in enumerate(reversed(base)):
            if i and not i % 3:
                digits.append(',')
            digits.append(c)
        base = ''.join(reversed(digits))
        return "${0}.{1}".format(base, cents)
    

In [6]:
#Now that we have the formatters set up, we just need to create the formatter factories:
class USAFormatterFactory:
    def create_date_formatter(self):
        return USADateFormatter()
    def create_currency_formatter(self):
        return USACurrencyFormatter()

class FranceFormatterFactory:
    def create_date_formatter(self):
        return FranceDateFormatter()
    def create_currency_formatter(self):
        return FranceCurrencyFormatter()


In [7]:
# a module-level variable:
country_code = 'US'
factory_map = {"US": USAFormatterFactory, "FR": FranceFormatterFactory}
formatter_factory = factory_map.get(country_code)()

# The composite pattern

In [16]:
class Folder:
    def __init__(self, name):
        self.name = name
        self.children = {}
        
    def add_child(self, child):
        pass
    
    def move(self, new_path):
        pass
    
    def copy(self, new_path):
        pass
    
    def delete(self):
        pass
    
class File:
    def __init__(self, name, contents):
        self.name = name
        self.contents = contents
    
    def move(self, new_path):
        pass
    
    def copy(self, new_path):
        pass
    
    def delete(self):
        pass

    

In [21]:
#To take advantage of the similar operations, 
# we can extract some of the common methods into a parent class
class Component:
    def __init__(self, name):
        self.name = name
    
    def move(self, new_path):
        new_folder = get_path(new_path)
        del self.parent.children[self.name]
        new_folder.children[self.name] = self
        self.parent = new_folder
        
    def delete(self):
        del self.parent.children[self.name]

class Folder(Component):
    def __init__(self, name):
        super().__init__(name)
        self.children = {}
    
    def add_child(self, child):
        child.parent = self
        self.children[child.name] = child
    
    def copy(self, new_path):
        pass

class File(Component):
    def __init__(self, name, contents):
        super().__init__(name)
        self.contents = contents
    
    def copy(self, new_path):
        pass
    
root = Folder('')
def get_path(path):
    names = path.split('\\')[1:]
    node = root
    for name in names:
        node = node.children[name]
    return node

        

In [23]:
folder1 = Folder('folder1')
folder2 = Folder('folder2')
root.add_child(folder1)
root.add_child(folder2) 
folder11 = Folder('folder11')
folder1.add_child(folder11)
file111 = File('file111', 'contents')
folder11.add_child(file111)
file21 = File('file21', 'other contents') 
folder2.add_child(file21)
folder2.children 


{'file21': <__main__.File at 0x2516563ef28>}

In [24]:
folder2.move('\\folder1\\folder11')

In [25]:
folder11.children

{'file111': <__main__.File at 0x2516563efd0>,
 'folder2': <__main__.Folder at 0x2516563eef0>}

In [27]:
file21.move('\\folder1')

In [28]:
folder1.children

{'file21': <__main__.File at 0x2516563ef28>,
 'folder11': <__main__.Folder at 0x2516563ec88>}

In [30]:
get_path('\\folder1\\folder11').children

{'file111': <__main__.File at 0x2516563efd0>,
 'folder2': <__main__.Folder at 0x2516563eef0>}